In [1]:
import pandas as pd
import numpy as np
import glob
import pandas_read_xml as pdx
from tqdm import tqdm
import base64

C:\Users\USER\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
xml_normal = glob.glob('./data/validation/normal/*.xml')
xml_arr = glob.glob('./data/validation/arrhythmia/*.xml')
xml_valid = xml_normal + xml_arr

In [4]:
valid_array = np.empty((0,8,1500))

In [72]:
rhythm_Lead

,LeadByteCountTotal,LeadTimeOffset,LeadSampleCountTotal,LeadAmplitudeUnitsPerBit,LeadAmplitudeUnits,LeadHighLimit,LeadLowLimit,LeadID,LeadOffsetFirstSample,FirstSampleBaseline,LeadSampleSize,LeadOff,BaselineSway,ExcessiveACNoise,MuscleNoise,LeadDataCRC32,WaveFormData
0,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,I,0,0,2,FALSE,TRUE,FALSE,FALSE,792309794,7P/s/+v/6//r/+v/6//r/+z/7P/s/+z/7P/s/+z/7P/r/+...
1,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,II,0,0,2,FALSE,TRUE,FALSE,FALSE,1992604133,CwALAAoACgANAA0ADQANAA0ADQANAA0ACwALAAsACwALAA...
2,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,V1,0,0,2,FALSE,TRUE,FALSE,FALSE,2479589297,8f/w//D/8P/y//L/8f/w//H/8f/x//L/9P/1//X/9f/1//...
3,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,V2,0,0,2,FALSE,TRUE,FALSE,FALSE,3867204371,AwADAAMAAwADAAIAAgAAAAIAAQABAAEA//////////////...
4,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,V3,0,0,2,FALSE,TRUE,FALSE,FALSE,4053991,yP/I/8j/yP/G/8b/xv/F/8X/xf/F/8T/xv/G/8b/xf/H/8...
5,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,V4,0,0,2,FALSE,TRUE,FALSE,FALSE,1255155516,2v/a/9r/2v/a/9r/2v/a/9r/2v/a/9r/3P/b/9v/2//b/9...
6,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,V5,0,0,2,FALSE,TRUE,FALSE,FALSE,2311898172,5//n/+f/5//n/+f/5//n/+f/5v/m/+b/5v/n/+f/5//n/+...
7,10000,0,5000,4.88,MICROVOLTS,2147483647,268435456,V6,0,0,2,FALSE,TRUE,FALSE,FALSE,2164440854,8//z//P/8//1//X/9f/1//P/8//z//P/8//y//L/8v/w//...


In [5]:

for xml in tqdm(xml_valid):
    
    Waveform = pdx.read_xml(xml, ['RestingECG', 'Waveform'])
    Rhythm = Waveform[1]
    df_rhythm = pd.json_normalize(Rhythm)    
    rhythm_Lead = pd.json_normalize(df_rhythm['LeadData'])
    df_RL = pd.DataFrame()
    
    for i in range(len(rhythm_Lead.columns)):
        df_normal = pd.json_normalize(rhythm_Lead[i])
        df_RL = df_RL.append(df_normal)  
    #rhythm_Lead = rhythm_Lead.drop([2,3,4,5])
        
    df_rhythm_sum = pd.concat([df_rhythm.iloc[:,:-1],df_RL],1)
    df_rhythm_sum = df_RL
    df_rhythm_sum.reset_index(drop=True,inplace=True)
    x = np.array(0)
    rhythm_dataset = np.empty((0,1500))
    
    for i in range(len(df_rhythm_sum['WaveFormData'])):
        wave_row = df_rhythm_sum.iloc[i,-1].replace('\n','').replace('\\n', '').replace('\\','').replace("b'",'')
        wave_row_bytes = base64.b64decode(wave_row)
        encoded_vec = np.fromstring(wave_row_bytes, dtype = np.int16)
        rhythm_dataset = np.vstack((rhythm_dataset,encoded_vec[2000:3500]))
    rhythm_dataset = rhythm_dataset.reshape(-1,8,1500)
    
    valid_array = np.vstack((valid_array,rhythm_dataset))

  0%|                                                                                         | 0/4859 [00:00<?, ?it/s]<ipython-input-5-8919a0786e71>:14: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_rhythm_sum = pd.concat([df_rhythm.iloc[:,:-1],df_RL],1)
<ipython-input-5-8919a0786e71>:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  encoded_vec = np.fromstring(wave_row_bytes, dtype = np.int16)
 54%|██████████████████████████████████████████▏                                   | 2630/4859 [03:02<02:34, 14.44it/s]


KeyError: 1

In [88]:
valid_array.shape

(2630, 8, 3000)

In [81]:
normal_valid_array = valid_array

In [6]:
#비정상일 때는 다른코드
for xml in tqdm(xml_valid[2630:]):
    
    Waveform = pdx.read_xml(xml, ['RestingECG', 'Waveform'])
    #df_rhythm = pd.json_normalize(Rhythm)
    rhythm_Lead = pd.json_normalize(Waveform['LeadData'][0])
    #df_RL = pd.DataFrame()
    
    #for i in range(len(rhythm_Lead.columns)):
    #    df_normal = pd.json_normalize(rhythm_Lead[i])
    #    df_RL = df_RL.append(df_normal)  
    #rhythm_Lead = rhythm_Lead.drop([2,3,4,5])
    df_RL = rhythm_Lead
        
    #df_rhythm_sum = pd.concat([df_rhythm.iloc[:,:-1],df_RL],1)
    df_rhythm_sum = df_RL
    df_rhythm_sum.reset_index(drop=True,inplace=True)
    x = np.array(0)
    rhythm_dataset = np.empty((0,1500))
    
    for i in range(len(df_rhythm_sum['WaveFormData'])):
        wave_row = df_rhythm_sum.iloc[i,-1].replace('\n','').replace('\\n', '').replace('\\','').replace("b'",'')
        wave_row_bytes = base64.b64decode(wave_row)
        encoded_vec = np.fromstring(wave_row_bytes, dtype = np.int16)
        rhythm_dataset = np.vstack((rhythm_dataset,encoded_vec[2000:3500]))
    rhythm_dataset = rhythm_dataset.reshape(-1,8,1500)
    
    valid_array = np.vstack((valid_array,rhythm_dataset))

  0%|                                                                                         | 0/2229 [00:00<?, ?it/s]<ipython-input-6-3f15ffc3b22a>:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  encoded_vec = np.fromstring(wave_row_bytes, dtype = np.int16)
100%|██████████████████████████████████████████████████████████████████████████████| 2229/2229 [05:56<00:00,  6.25it/s]


In [90]:
valid_array.shape

(4859, 8, 3000)

In [7]:
arr_reshaped = valid_array.reshape(valid_array.shape[0], -1)

In [8]:
np.save('./x_save_valid', arr_reshaped) # x_save.npy

In [92]:
np.savetxt('valid_array_4859.txt',arr_reshaped)

In [38]:
json_label_normal = glob.glob('./label/train/normal/*.json')

In [39]:
json_label_normal

['./label/train/normal\\5_0_000001_ecg.json',
 './label/train/normal\\5_0_000002_ecg.json',
 './label/train/normal\\5_0_000003_ecg.json',
 './label/train/normal\\5_0_000004_ecg.json',
 './label/train/normal\\5_0_000005_ecg.json',
 './label/train/normal\\5_0_000006_ecg.json',
 './label/train/normal\\5_0_000007_ecg.json',
 './label/train/normal\\5_0_000008_ecg.json',
 './label/train/normal\\5_0_000009_ecg.json',
 './label/train/normal\\5_0_000010_ecg.json',
 './label/train/normal\\5_0_000011_ecg.json',
 './label/train/normal\\5_0_000012_ecg.json',
 './label/train/normal\\5_0_000013_ecg.json',
 './label/train/normal\\5_0_000014_ecg.json',
 './label/train/normal\\5_0_000015_ecg.json',
 './label/train/normal\\5_0_000016_ecg.json',
 './label/train/normal\\5_0_000017_ecg.json',
 './label/train/normal\\5_0_000018_ecg.json',
 './label/train/normal\\5_0_000019_ecg.json',
 './label/train/normal\\5_0_000020_ecg.json',
 './label/train/normal\\5_0_000021_ecg.json',
 './label/train/normal\\5_0_000022

In [60]:
l = np.zeros((21037))

In [63]:
k = np.ones(17835)

In [57]:
n = np.hstack((l,k))

In [59]:
n.shape

(38872,)

In [64]:
l = np.zeros(2630)
k = np.ones(2229)
valid_label = np.hstack((l,k))
l = np.zeros((21037))
k = np.ones(17835)
train_label = np.hstack((l,k))


In [65]:
train_label

array([0., 0., 0., ..., 1., 1., 1.])

In [66]:
valid_label

array([0., 0., 0., ..., 1., 1., 1.])